<a href="https://colab.research.google.com/github/anorlo/Pytorch/blob/master/H_DenseUNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
path = "/content/drive/My Drive/H-DenseUNet-master"

os.chdir(path)
os.listdir(path)

['bash_train.sh',
 'train_2ddense.py',
 'denseunet.py',
 '.gitignore',
 'train_hybrid.py',
 'hybridnet.py',
 'test.py',
 'requirements.txt',
 'README.md',
 'denseunet3d.py',
 'loss.py',
 'densenet.py',
 '.idea',
 'lib',
 'Keras-2.0.8',
 'preprocessing.py',
 'data']

In [15]:
!python preprocessing.py

  File "preprocessing.py", line 17
    img = np.array(img, dtype='float32')
                                       ^
TabError: inconsistent use of tabs and spaces in indentation


In [3]:
!pip install medpy

     |████████████████████████████████| 153kB 8.1MB/s 
     |████████████████████████████████| 47.4MB 67kB/s 
  Created wheel for medpy: filename=MedPy-0.4.0-cp36-cp36m-linux_x86_64.whl size=753441 sha256=f024bbbdace8f73d5552cfea5ba5e4f4891e360615c8248f8db8bbb4d2b66b9f
  Stored in directory: /root/.cache/pip/wheels/8c/c9/9c/2c6281c7a72b9fb1570862a4f028af7ce38405008354fbf870
Successfully built medpy


In [ ]:
from medpy.io import load, save
import os
import os.path
import numpy as np


def proprecessing(image_path, save_folder):

    if not os.path.exists("data/"+save_folder):
        os.mkdir("data/"+save_folder)
    filelist = os.listdir(image_path)
    filelist = [item for item in filelist if 'volume' in item]
    for file in filelist:
        img, img_header = load(image_path+file)
        img[img < -200] = -200
        img[img > 250] = 250
    img = np.array(img, dtype='float32')
    print ("Saving image "+file)
    save(img, "./data/" + save_folder + file)

def generate_livertxt(image_path, save_folder):
    if not os.path.exists("data/"+save_folder):
        os.mkdir("data/"+save_folder)

    # Generate Livertxt
    if not os.path.exists("data/"+save_folder+'LiverPixels'):
        os.mkdir("data/"+save_folder+'LiverPixels')

    for i in range(0,131):
        livertumor, header = load(image_path+'segmentation-'+str(i)+'.nii')
        f = open('data/' +save_folder+'/LiverPixels/liver_' + str(i) + '.txt', 'w')
        index = np.where(livertumor==1)
        x = index[0]
        y = index[1]
        z = index[2]
        np.savetxt(f, np.c_[x,y,z], fmt="%d")
	
        f.write("\n")
        f.close()

def generate_tumortxt(image_path, save_folder):
    if not os.path.exists("data/"+save_folder):
        os.mkdir("data/"+save_folder)

    # Generate Livertxt
    if not os.path.exists("data/"+save_folder+'TumorPixels'):
        os.mkdir("data/"+save_folder+'TumorPixels')

    for i in range(0,131):
        livertumor, header = load(image_path+'segmentation-'+str(i)+'.nii')
        f = open("data/"+save_folder+"/TumorPixels/tumor_"+str(i)+'.txt','w')
        index = np.where(livertumor==2)

        x = index[0]
        y = index[1]
        z = index[2]

        np.savetxt(f,np.c_[x,y,z],fmt="%d")

        f.write("\n")
        f.close()

def generate_txt(image_path, save_folder):
    if not os.path.exists("data/"+save_folder):
        os.mkdir("data/"+save_folder)

    # Generate Livertxt
    if not os.path.exists("data/"+save_folder+'LiverBox'):
        os.mkdir("data/"+save_folder+'LiverBox')
    for i in range(0,131):
        values = np.loadtxt('data/myTrainingDataTxt/LiverPixels/liver_' + str(i) + '.txt', delimiter=' ', usecols=[0, 1, 2])
        a = np.min(values, axis=0)
        b = np.max(values, axis=0)
        box = np.append(a,b, axis=0)
        np.savetxt('data/myTrainingDataTxt/LiverBox/box_'+str(i)+'.txt', box,fmt='%d')


proprecessing(image_path='data/TrainingData/', save_folder='myTrainingData/')
proprecessing(image_path='data/TestData/', save_folder='myTestData/')
print ("Generate liver txt ")
generate_livertxt(image_path='data/TrainingData/', save_folder='myTrainingDataTxt/')
print ("Generate tumor txt")
generate_tumortxt(image_path='data/TrainingData/', save_folder='myTrainingDataTxt/')
print ("Generate liver box ")
generate_txt(image_path='data/TrainingData/', save_folder='myTrainingDataTxt/')
